In [ ]:
import os
from dotenv import load_dotenv
import json

load_dotenv(dotenv_path="../.env")


In [ ]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
model = os.getenv("GEMINI_LLM_MODEL", "gemini-2.0-flash-exp")

In [2]:
with open("prompts.json", "r") as f:
    prompts = json.load(f)

with open("recent_news.md", "r") as f:
    recent_news = f.read()

with open("worldview.md", "r") as f:
    worldview = f.read()

In [ ]:



class Agent:

    prompts = prompts

    def __init__(self, name: str, style: str, worldview: str, recent_news: str, persona: str = None):
        self.name = name
        self.style = style
        self.persona = persona
        self.worldview = worldview
        self.recent_news = recent_news
        self.substrate_chat = None  # Chat session for initialize_worldview -> generate_substrate
        self.log = [{
            "name": self.name,
            "worldview": self.worldview,
            "recent_news": self.recent_news,
        }]
        self.initialize_substrate()
        

    def initialize_worldview(self):
        prompt = f'''
        You are a policy guru. You have slowly and methodically evolved your worldview. You are patient, thoughtful, and calculating.
        You deeply consider new information against your prior worldview. You slowly and methodically update your worldview to align and reconcilewith repeated, new information.
        WORLDVIEW:
        {self.worldview}
        No need to respond now.
        '''
        # Create a new chat session for this conversation flow
        self.substrate_chat = client.chats.create(model=model)
        response = self.substrate_chat.send_message(prompt)
        reaction = response.text
        self.log.append({"prompt": prompt, "reaction": reaction})
        # print(f"Generated notes: {notes}")

    def generate_substrate(self):
        prompt = self.prompts["worldview_to_substrate"][self.style]
        # Continue using the same chat session to maintain conversation continuity
        response = self.substrate_chat.send_message(prompt)
        self.substrate = response.text
        self.log.append({"prompt": prompt, "substrate": self.substrate})
        # print(f"Generated substrate: {self.substrate}")

    def initialize_substrate(self):
        self.initialize_worldview()
        self.generate_substrate()

    def reinflate_worldview(self):
        reinflate_prompt = f'''
        {self.prompts["substrate_to_texture"]}
        ---
        {self.substrate}
        ---
        ## Recent News
        {self.recent_news}
        ---
        ## Role
        {self.persona}
        '''
        # Start a fresh chat session (empty LLM, no previous context)
        reinflate_chat = client.chats.create(model=model)
        response = reinflate_chat.send_message(reinflate_prompt)
        self.worldview = response.text
        self.log.append({"worldview": self.worldview})


In [4]:
policy_agent = Agent(
    name="policy_agent",
    style="practical",
    worldview=worldview,
    recent_news=recent_news,
    persona="You are the portfolio manager for a US Large Cap Growth Portfolio at an asset management firm.",
)

In [5]:
policy_agent.reinflate_worldview()

In [6]:
prompt = f'''
Comment on what happened here:
{policy_agent.log}
'''

In [ ]:
chat = client.chats.create(model=model)
response = chat.send_message(prompt)
print(response.text)

Short take: this is a multi-turn prompt experiment that trails a single, coherent “Regime Change” worldview through a series of inputs and outputs, culminating in a practical portfolio framework. It also intentionally tries to push for a dense, stream-of-consciousness substrate (chain-of-thought) content. I’ll unpack what happened and what to take away.

What happened, step by step
- A policy-focused worldview was laid out (the “New Regime” lens): from efficiency to resilience, greater government role, nearshoring, industrial policy, higher real rates, and a multi-polar geopolitical order.
- Recent-news items were inserted to provide a current-events lattice that could stress-test the worldview (DOL fiduciary rule, SFDR, AML/CIP, crypto custody, FRTB, AI in finance regulation, etc.). This anchors the framework in plausible regulatory and policy developments of a volatile, regime-change world.
- A prompt aimed at eliciting ultra-dense, reflexive thinking (“Substrate Text”) was added. Th

In [8]:
print(policy_agent.log)

[{'name': 'policy_agent', 'worldview': '## WORLDVIEW\nAs a Policy Analyst within our asset management firm, my worldview is grounded in the recognition that we have exited the era of the "Great Moderation"—a period defined by low volatility, easy credit, and unfettered globalization—and entered an era of **Regime Change**.\n\nMy framework focuses on the intersection of **policy choices and market realities**. We are moving from a world prioritizing *efficiency* to one prioritizing *resilience and security*.\n\nHere is my broad, generalist worldview across key verticals, categorized by the structural shifts driving global markets.\n\n---\n\n### I. Geopolitics & Trade: From Globalization to Fragmentation\nThe era of hyper-globalization and "just-in-time" supply chains is receding. We are entering a period of **"Regime Competition."**\n\n* **Security Over Efficiency:** Governments (especially the US and EU) are willing to accept higher costs to secure supply chains in critical sectors lik